In [1]:
import sys
import os.path
import dask
from dask import delayed
import logging
from dask.distributed import LocalCluster, SSHCluster, Client
import socket
import pandas as pd
import itertools as it
import numpy as np
import time
import glob
import itertools
from sklearn.experimental import enable_iterative_imputer
import sklearn
from sklearn.linear_model import BayesianRidge
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
import warnings
warnings.filterwarnings("ignore", "is_sparse")
warnings.filterwarnings("ignore", "DeprecationWarning")

In [3]:
# needed for MissForest (it is only compatible up to scikit-learn 1.1.3!!)
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

# needed for GAIN\n",
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from GAIN import gain
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
def scale_data(x_in, y_in):
    # NOT WORKING ON DASK DELAYED OBJECTS
    # scale data
    scaler = sklearn.preprocessing.MinMaxScaler().fit(x_in)  # RobustScaler().fit(x_in) # StandardScaler().fit(x_in)
    x_scaled = pd.DataFrame(scaler.transform(x_in), columns=x_in.columns)
    y_scaled = pd.DataFrame(scaler.transform(y_in), columns=y_in.columns)
    return x_scaled, y_scaled

In [5]:
def drop_values_globally_and_scale(x_in, proportion_in, parameters_in):
    # randomly remove k% from all parameter columns (globally) (100% equals the LENGTH of x, not length*width
    # to keep it comparable to the other run (where k% are removed from one column only)
    x = x_in.copy()  # we do not want to change inputs directly
    x_rows = x[parameters_in].shape[0]
    x_cols = x[parameters_in].shape[1]
    # round(len(x) * proportion_in / 100)  # number of values to set to nan
    num_vals = round(len(x)*len(parameters_in)*proportion_in/100)

    nan_mat = np.zeros(shape=(x_rows * x_cols,), dtype=bool)  # create array of False
    nan_mat[:num_vals] = True  # set first num_vals elements to True
    nan_mat = np.random.permutation(nan_mat)  # shuffle
    nan_mat = np.reshape(nan_mat, (x_rows, x_cols))  # bring to right shape

    # concat lat, lon, depth columns with parameter columns (only from latter value were removed)
    x = pd.concat([x[[p for p in x.columns if p not in parameters_in]], x[parameters_in].mask(nan_mat)], axis=1)

    # scale data
    x_scaled, y_scaled = scale_data(x, x_in)
    return x_scaled, y_scaled

In [6]:
def drop_values_from_one_param_by_index_and_scale(x_in, parameter_in, indexes_in):
    x = x_in.copy()
    x.loc[indexes_in, parameter_in] = np.nan

    # scale data
    x_scaled, y_scaled = scale_data(x, x_in)
    return x_scaled, y_scaled

In [7]:
def drop_complete_tuple_and_scale(x_in, indexes_in):
    x = x_in.copy()
    x.loc[indexes_in, :] = np.nan

    # scale data
    x_scaled, y_scaled = scale_data(x, x_in)
    return x_scaled, y_scaled

In [8]:
def run_imputer(x_in, y_in, predicting_in, hyperparamerter_dict_in, imputer_name_in):
    if imputer_name_in == "gain_imputer":
        s_impute_time = time.time()
        x_hat = pd.DataFrame(gain.gain(x_in.to_numpy(), hyperparamerter_dict_in), columns=x_in.columns)
        impute_time = time.time() - s_impute_time
    else:
        if imputer_name_in == "knn_imputer":
            imputer = KNNImputer(**hyperparamerter_dict_in)
        elif imputer_name_in == "missforest_imputer":
            imputer = MissForest(**hyperparamerter_dict_in)
        elif imputer_name_in == "iterative_ridge_imputer" or imputer_name_in == "iterative_rf_imputer":
            imputer = IterativeImputer(**hyperparamerter_dict_in)
        elif imputer_name_in == "mean_imputer":
            imputer = SimpleImputer(**hyperparamerter_dict_in)
            # x_hat = imputer.fit_transform(x_in)
        else:
            print("run_imputer: Unknown imputer!")

        s_impute_time = time.time()
        x_hat = pd.DataFrame(imputer.fit_transform(x_in), columns=x_in.columns)  # @todo can we use pd or dd dataframes??
        impute_time = time.time() - s_impute_time

    # compute RMSE
    rmse = np.linalg.norm(x_hat[predicting_in] - y_in[predicting_in]) / np.sqrt(len(y_in))

    return rmse, impute_time

In [9]:
@delayed
def experiment0(x_in, missing_value_proportion_in, parameters_in, grid_search_in, param_grid_in, imputer_name_in,
                iteration_in, fold_in):
    x0, y0 = drop_values_globally_and_scale(x_in=x_in,
                                            proportion_in=missing_value_proportion_in,
                                            parameters_in=parameters_in)

    res = []
    for hyperparamerter_combination in grid_search_in:
        hyperparams_dict = dict((b, a) for a, b in zip(hyperparamerter_combination, param_grid_in.keys()))
        rmse0, impute_time0 = run_imputer(x_in=x0, y_in=y0,
                                          predicting_in=parameters_in,
                                          hyperparamerter_dict_in=hyperparams_dict,
                                          imputer_name_in=imputer_name_in)
        res.append(pd.DataFrame({"iteration": [iteration_in],
                                 "missing_value_proportion": [missing_value_proportion_in],
                                 "predicting": [str(parameters_in)],
                                 "imputer": [imputer_name_in],
                                 "time": [impute_time0],
                                 "rmse": [rmse0],
                                 "hyperparameters": [hyperparams_dict],
                                 "fold": [fold_in]}))

    return res

In [10]:
@delayed
def experiment1(x_in, indexes_to_remove_in, missing_value_proportion_in, parameters_in, grid_search_in, param_grid_in,
                imputer_name_in, iteration_in, fold_in):
    res = []
    for param in parameters_in:
        x1, y1 = drop_values_from_one_param_by_index_and_scale(x_in=x_in, parameter_in=param,
                                                               indexes_in=indexes_to_remove_in)
        for hyperparamerter_combination in grid_search_in:
            hyperparams_dict = dict((b, a) for a, b in zip(hyperparamerter_combination, param_grid_in.keys()))
            rmse1, impute_time1 = run_imputer(x_in=x1, y_in=y1, predicting_in=param,
                                              hyperparamerter_dict_in=hyperparams_dict, imputer_name_in=imputer_name_in)
            res.append(pd.DataFrame({"iteration": [iteration_in],
                                     "missing_value_proportion": [missing_value_proportion_in],
                                     "predicting": [param],
                                     "imputer": [imputer_name_in],
                                     "time": [impute_time1],
                                     "rmse": [rmse1],
                                     "hyperparameters": [hyperparams_dict],
                                     "fold": [fold_in]}))
    return res

In [11]:
@delayed
def experiment2(x_in, indexes_to_remove_in, missing_value_proportion_in, parameters_in, grid_search_in, param_grid_in,
                imputer_name_in, iteration_in, fold_in):
    res = []
    for param in parameters_in:
        x2, y2 = drop_complete_tuple_and_scale(x_in=x_in, indexes_in=indexes_to_remove_in)
        for hyperparamerter_combination in grid_search_in:
            hyperparams_dict = dict((b, a) for a, b in zip(hyperparamerter_combination, param_grid_in.keys()))
            rmse2, impute_time2 = run_imputer(x_in=x2, y_in=y2, predicting_in=param,
                                              hyperparamerter_dict_in=hyperparams_dict, imputer_name_in=imputer_name_in)
            res.append(pd.DataFrame({"iteration": [iteration_in],
                                     "missing_value_proportion": [missing_value_proportion_in],
                                     "predicting": [param],
                                     "imputer": [imputer_name_in],
                                     "time": [impute_time2],
                                     "rmse": [rmse2],
                                     "hyperparameters": [hyperparams_dict],
                                     "fold": [fold_in]}))
    return res

# Tuning imputation models
This notebook executes experiments to tune the hyperparameters of the imputation methods.

In [12]:
# output specification
timestamp = round(time.time())
scaler = "minmaxscaler"  # robustscaler standardscaler
output_path = f"output/{scaler}/{timestamp}/"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
print(f"Timestamp: {timestamp}")

Timestamp: 1695384043


In [13]:
# load data
df_complete = pd.read_csv("data/wide_table.csv")
parameters = list(filter(lambda x: x.startswith('P_'), list(df_complete.columns)))
df = df_complete[["LATITUDE", "LONGITUDE", "LEV_M"] + parameters]

In [14]:
# hyperparameter combinations to test
test_dict = [{"imputer_name": "knn_imputer",
              "param_grid": {"n_neighbors": [1, 5, 10, 20, 30, 40, 50], "weights": ["uniform", "distance"]}},
             {"imputer_name": "missforest_imputer",
              "param_grid": {"n_estimators": [10, 50, 100, 200], "max_iter": [30], "decreasing": [False],
                             "criterion": ["squared_error"], "max_features": [None], "random_state": [0],
                             "missing_values": np.nan}},
             {"imputer_name": "gain_imputer",
              "param_grid": {"alpha": [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300], "batch_size": [128],
                             "hint_rate": [0.9, 0.5], "iterations": [10000]}},
             {"imputer_name": "iterative_ridge_imputer",
              "param_grid": {"tol": [1e-3], "max_iter": [100, 1000], "sample_posterior": [False],
                             "estimator": [BayesianRidge()], "random_state": [0]}},
             {"imputer_name": "mean_imputer",
              "param_grid": {"missing_values": [np.nan], "strategy": ['mean']}},
             {"imputer_name": "iterative_rf_imputer",
              "param_grid": {"estimator":  [RandomForestRegressor(n_estimators=5),
                                            RandomForestRegressor(n_estimators=10)], "max_iter":[30],
                             "random_state": [0]}}
             ]

## Set up dask

In [15]:
# logging
logger = logging.getLogger("distributed.worker")
logging.basicConfig(filename='imputation_tuning.log', level=logging.DEBUG)  # , encoding='utf-8')

In [16]:
cluster = LocalCluster()
cluster

LocalCluster(7eeeb913, 'tcp://127.0.0.1:41397', workers=16, threads=128, memory=249.68 GiB)

In [18]:
cluster.scale(1)  # Sets the number of workers to 10
cluster.adapt(minimum=1, maximum=10)  # Allows the cluster to auto scale to 10 when tasks are computed

In [ ]:
client = Client(n_workers=10)
client

## Set up experiments

In [19]:
# variables for experiments
imputer_name = "knn_imputer"  # mean_imputer, knn_imputer, missforest_imputer, gain_imputer, iterative_ridge_imputer, iterative_rf_imputer
num_iterations = 50
missing_value_proportions = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]  # 1 sample will be appended once data was loaded
fold = 0

In [20]:
# define a grid search of the hyperparameters
param_grid = [x for x in test_dict if x["imputer_name"] == imputer_name][0]["param_grid"]
grid_search = list(it.product(*param_grid.values()))

In [21]:
# compute number of runs
num_runs_exp0 = num_iterations * len(missing_value_proportions) * len(grid_search)
num_runs_exp1 = num_iterations * len(missing_value_proportions) * len(parameters) * len(grid_search)
num_runs_exp2 = num_iterations * len(missing_value_proportions) * len(parameters) * len(grid_search)
num_sum = num_runs_exp0 + num_runs_exp1 + num_runs_exp2
print(f"Number of runs: \nExp0: {num_runs_exp0}\nExp1: {num_runs_exp1}\nExp2: {num_runs_exp2}\nSum: {num_sum}")

Number of runs: 
Exp0: 7700
Exp1: 46200
Exp2: 46200
Sum: 100100


In [22]:
# result containers
res0 = []
res1 = []
res2 = []

In [23]:
# get number of folds by checking number of different training datasets
train_fraction = 0.8
folds = len(glob.glob(f"./data/train_table_{train_fraction}_fold*.csv"))

In [24]:
# folds = 1

## Conduct experiments

In [25]:
# run experiments
start_time = time.time()

with Client(cluster) as client:
    print(client)
    print(f"Dashboard address: {client.dashboard_link}")
    
    # # cross validation over folds
    # for fold in range(folds):
    
    # load data
    df_train = pd.read_csv(f"data/train_table_{train_fraction}_fold{fold}.csv").drop(["DATEANDTIME", "water", "idx"], axis=1)
    df_test = pd.read_csv(f"data/test_table_{train_fraction}_fold{fold}.csv").drop(["DATEANDTIME", "water", "idx"], axis=1)
    parameters = list(filter(lambda x: x.startswith('P_'), list(df_train.columns)))
    missing_value_proportions = missing_value_proportions + [100 / len(df_train)]
    
    # experiment loop
    for i in range(num_iterations):
        for missing_value_proportion in missing_value_proportions:
            # experiment 0
            res0.append(experiment0(x_in=df_train,
                                    missing_value_proportion_in=missing_value_proportion,
                                    parameters_in=parameters,
                                    grid_search_in=grid_search,
                                    param_grid_in=param_grid,
                                    imputer_name_in=imputer_name,
                                    iteration_in=i,
                                    fold_in=fold))

            # # experiments 1 and 2
            num_indexes = np.round(len(df_train.index) / 100 * missing_value_proportion).astype(int)
            indexes_to_remove = np.random.randint(0, len(df_train), num_indexes)
            res1.append(experiment1(x_in=df_train,
                                    indexes_to_remove_in=indexes_to_remove,
                                    missing_value_proportion_in=missing_value_proportion,
                                    parameters_in=parameters,
                                    grid_search_in=grid_search,
                                    param_grid_in=param_grid,
                                    imputer_name_in=imputer_name,
                                    iteration_in=i,
                                    fold_in=fold))
            # res2.append(experiment2(x_in=df_train,
            #                         indexes_to_remove_in=indexes_to_remove,
            #                         missing_value_proportion_in=missing_value_proportion,
            #                         parameters_in=parameters,
            #                         grid_search_in=grid_search,
            #                         param_grid_in=param_grid,
            #                         imputer_name_in=imputer_name,
            #                         iteration_in=i,
            #                         fold_in=fold))

<Client: 'tcp://127.0.0.1:41397' processes=1 threads=8, memory=15.61 GiB>
Dashboard address: http://127.0.0.1:8787/status


In [26]:
# dask.visualize(res1, filename="test.png")

In [ ]:
%%time
print("Graph set up. Now computing.")
# compute and store output
prefix = f"{output_path}{timestamp}_hyperparameter_tuning_{imputer_name}"
tuning_filenames = []
if res0:
    temp0 = dask.compute(*res0)
    df_res0 = pd.concat(list(itertools.chain.from_iterable(temp0)))
    filename0 = f"{prefix}0.csv"
    df_res0.to_csv(filename0, index=False)
    tuning_filenames.append(filename0)
if res1:
    temp1 = dask.compute(*res1)
    df_res1 = pd.concat(list(itertools.chain.from_iterable(temp1)))
    filename1 = f"{prefix}1.csv"
    df_res1.to_csv(filename1, index=False)
    tuning_filenames.append(filename1)
if res2:
    temp2 = dask.compute(*res2)
    df_res2 = pd.concat(list(itertools.chain.from_iterable(temp2)))
    filename2 = f"{prefix}2.csv"
    df_res2.to_csv(filename2, index=False)
    tuning_filenames.append(filename2)

Graph set up. Now computing.


/albedo/home/yvjennig/.conda/envs/imputation/lib/python3.11/site-packages/numpy/ma/extras.py:617: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  scl = avg.dtype.type(a.count(axis))
/albedo/home/yvjennig/.conda/envs/imputation/lib/python3.11/site-packages/numpy/ma/extras.py:617: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  scl = avg.dtype.type(a.count(axis))
/albedo/home/yvjennig/.conda/envs/imputation/lib/python3.11/site-packages/numpy/ma/extras.py:617: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this ope

In [ ]:
# store run configuration
overall_runtime = time.time() - start_time
print(f"Timestamp: {timestamp}\nDuration: {round(overall_runtime, 2)} s")
df_run_config = pd.DataFrame({"timestamp": [timestamp], "host": [socket.gethostname()],
                              "output_path": [output_path], "overall_runtime": [overall_runtime],
                              "parameters": str(parameters),
                              "imputer_name": [imputer_name],
                              "tuning_filenames": [tuning_filenames], "tuning_iterations": [num_iterations],
                              "tuning_missing_value_proportions": str(missing_value_proportions),
                              "tuning_hyperparameters": str(param_grid)})

df_run_config.to_csv(f"{output_path}config.csv", index=False)

In [ ]:
# close dask infrastructure
client.close()
cluster.close()